In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("xsum")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.2 MB/s eta 0:00:00


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
text_column = dataset['test']
print(text_column)

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11334
})


In [ ]:
dataset['test'][11156]['document']

'West Midlands Police said the man had been detained in Reading and the woman in Wokingham by counter-terrorism officers at about 19:30 BST.\nThe pair, believed to be friends, were taken to a police station in the West Midlands for questioning.\nThe force said the arrests had been intelligence-led.\nThe suspects did not pose any imminent threat, a spokesperson added.\nSupt Robin Rickard from Thames Valley Police said the arrests "were part of an ongoing investigation by West Midlands Counter Terrorism Unit and were pre-planned".\n"Officers from Reading and Wokingham are providing support and there are searches taking place at the suspects\' home addresses," he said.'

In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create a folder in the root directory
#!mkdir -p "/content/drive/My Drive/XSum Dataset"

Mounted at /content/drive


In [ ]:
text_column.to_csv("/content/drive/My Drive/XSum Dataset/xsum_test.csv", index = None, header=True)

In [ ]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv(r'/content/drive/My Drive/XSum Dataset/xsum_test.csv')
df_1 = df[:1000]
df_2 = df[1000:2000]
df_3 = df[2000:3000]
df_4 = df[3000:4000]
df_5 = df[4000:5000]
df_6 = df[5000:6000]
df_7 = df[6000:7000]
df_8 = df[7000:8000]
df_9 = df[8000:9000]
df_10 = df[9000:10000]
df_11 = df[10000:11000]
df_12 = df[11000:]
df_3

In [ ]:
test_column = df_12['document']
file_directory = '/content/drive/My Drive/XSum Dataset/xsum_test_textranked_12.csv'

In [ ]:
n=10999
for x in test_column:
  n = n + 1
  text = x
  if pd.isnull(df_12.loc[n, 'document']):
    continue

  #Tokenizing the text
  sentences=sent_tokenize(text)

  #Removing Stopwords
  sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
  stop_words = stopwords.words('english')
  sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

  #Embedding with word2vec
  w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)
  sentence_embeddings=[[w2v.wv[word][0] for word in words] for words in sentence_tokens]
  max_len=max([len(tokens) for tokens in sentence_tokens])
  sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

  #Calculating Simularity using Cosine Simularity
  similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
  for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
      similarity_matrix[i][j]=abs(1-spatial.distance.cosine(row_embedding,column_embedding))

  #Creating Graph
  nx_graph = nx.from_numpy_array(similarity_matrix)
  scores = nx.pagerank(nx_graph)
  top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
  top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
  print("<<<text " + str(n+1) + " summarized>>>")
  temp_summary_list = []
  for sent in sentences:
    if sent in top.keys():
      temp_summary_list.append(sent)
  summary=str(temp_summary_list)
  summary=summary.replace("[","")
  summary=summary.replace("]","")
  summary=summary.replace(","," ")
  summary=summary.replace("\'","")
  df_12.loc[n, 'textrank'] = summary

df_12.to_csv(file_directory)

df_12.head()


<<<text 11001 summarized>>>


<ipython-input-13-33ebd8e62e7b>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_12.loc[n, 'textrank'] = summary


<<<text 11002 summarized>>>
<<<text 11003 summarized>>>
<<<text 11004 summarized>>>
<<<text 11005 summarized>>>
<<<text 11006 summarized>>>
<<<text 11007 summarized>>>
<<<text 11008 summarized>>>
<<<text 11009 summarized>>>
<<<text 11010 summarized>>>
<<<text 11011 summarized>>>
<<<text 11012 summarized>>>
<<<text 11013 summarized>>>
<<<text 11014 summarized>>>
<<<text 11015 summarized>>>
<<<text 11016 summarized>>>
<<<text 11017 summarized>>>
<<<text 11018 summarized>>>
<<<text 11019 summarized>>>
<<<text 11020 summarized>>>
<<<text 11021 summarized>>>
<<<text 11022 summarized>>>
<<<text 11023 summarized>>>
<<<text 11024 summarized>>>
<<<text 11025 summarized>>>
<<<text 11026 summarized>>>
<<<text 11027 summarized>>>
<<<text 11028 summarized>>>
<<<text 11029 summarized>>>
<<<text 11030 summarized>>>
<<<text 11031 summarized>>>
<<<text 11032 summarized>>>
<<<text 11033 summarized>>>
<<<text 11034 summarized>>>
<<<text 11035 summarized>>>
<<<text 11036 summarized>>>
<<<text 11037 summar

/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


<<<text 11097 summarized>>>
<<<text 11098 summarized>>>
<<<text 11099 summarized>>>
<<<text 11100 summarized>>>
<<<text 11101 summarized>>>
<<<text 11102 summarized>>>
<<<text 11103 summarized>>>
<<<text 11104 summarized>>>
<<<text 11105 summarized>>>
<<<text 11106 summarized>>>
<<<text 11107 summarized>>>
<<<text 11108 summarized>>>
<<<text 11109 summarized>>>
<<<text 11110 summarized>>>
<<<text 11111 summarized>>>
<<<text 11112 summarized>>>
<<<text 11113 summarized>>>
<<<text 11114 summarized>>>
<<<text 11115 summarized>>>
<<<text 11116 summarized>>>
<<<text 11117 summarized>>>
<<<text 11118 summarized>>>
<<<text 11119 summarized>>>
<<<text 11120 summarized>>>
<<<text 11121 summarized>>>
<<<text 11122 summarized>>>
<<<text 11123 summarized>>>
<<<text 11124 summarized>>>
<<<text 11125 summarized>>>
<<<text 11126 summarized>>>
<<<text 11127 summarized>>>
<<<text 11128 summarized>>>
<<<text 11129 summarized>>>
<<<text 11130 summarized>>>
<<<text 11131 summarized>>>
<<<text 11132 summar

,document,summary,id,textrank
11000,He had back surgery for the second time in 18 ...,Tiger Woods is back in upbeat mood and expecti...,35164864,He is now ranked 414th and won the last of his...
11001,Real Madrid's Gareth Bale will hope to prove h...,"In three weeks, the eyes of the sporting world...",39908649,"""I hope Gareth plays because it would be great..."
11002,It represents one of the biggest tie-ups betwe...,The National Basketball Association has teamed...,38918505,In England Premier League clubs Manchester Ci...
11003,The Sunday Times says the missile veered off c...,Theresa May is coming under pressure to say wh...,38711418,Questioned by Andrew Marr the PM refused to s...
11004,"It goes on. A first Olympic diving gold, a fir...",This has been the era where British sport has ...,37882468,"""And you feel like you\re disengaged not only..."


In [ ]:
pd.isnull(df.loc[100, 'textrank'])

True

In [ ]:
df_2.head()

,document,summary,id,textrank
1001,Fifa delegates are voting on their new preside...,David Cameron has said Fifa president Sepp Bla...,32931356,"""It comes as the UKs Serious Fraud Office (SFO..."
1002,"The Romanian diners, who had paid a deposit of...",Some 120 diners celebrating a baptism at a res...,39139784,"""It happened in the space of a minute "" he sai..."
1003,There were 70 reports during the lambing seaso...,Reported incidents of livestock worrying have ...,36706313,Police Scotland said the crime which involved...
